In [49]:
%%writefile omp_stats.cpp
#include <iostream>        // Я подключаю ввод-вывод
#include <vector>          // Я использую vector
#include <omp.h>           // Я подключаю OpenMP
#include <cmath>           // Я подключаю sqrt()

using namespace std;

int main() {
    const int N = 10000000;               // Я задаю размер массива
    vector<double> data(N, 1.0);          // Я создаю массив из единиц

    double t_start = omp_get_wtime();     // Я начинаю общий замер времени

    double sum = 0.0;                     // Я объявляю сумму
    #pragma omp parallel for reduction(+:sum)
    for (int i = 0; i < N; i++) {
        sum += data[i];                   // Я считаю сумму параллельно
    }

    double mean = sum / N;                // Я считаю среднее

    double variance = 0.0;                // Я объявляю дисперсию
    #pragma omp parallel for reduction(+:variance)
    for (int i = 0; i < N; i++) {
        variance += (data[i] - mean) * (data[i] - mean); // Я считаю дисперсию
    }
    variance /= N;

    double t_end = omp_get_wtime();        // Я заканчиваю замер времени

    cout << "Threads: " << omp_get_max_threads() << endl;
    cout << "Mean: " << mean << endl;
    cout << "Variance: " << variance << endl;
    cout << "Time: " << (t_end - t_start) << " s" << endl;

    return 0;
}



Writing omp_stats.cpp


In [50]:
!g++ omp_stats.cpp -fopenmp -O2 -o omp_stats
!./omp_stats


Threads: 2
Mean: 1
Variance: 0
Time: 0.0149982 s


In [51]:
%%writefile memory_access.cu
#include <iostream>            // Я подключаю ввод-вывод
#include <cuda_runtime.h>      // Я подключаю CUDA

__global__ void coalesced(float* data) {
    int i = blockIdx.x * blockDim.x + threadIdx.x; // Я обращаюсь к памяти линейно
    if (i < 1024*1024) data[i] *= 2.0f;
}

__global__ void non_coalesced(float* data) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int idx = (i * 32) % (1024*1024);               // Я создаю разрозненный доступ
    data[idx] *= 2.0f;
}

int main() {
    const int N = 1024 * 1024;
    size_t bytes = N * sizeof(float);

    float* h = new float[N];
    for (int i = 0; i < N; i++) h[i] = 1.0f;

    float* d;
    cudaMalloc(&d, bytes);
    cudaMemcpy(d, h, bytes, cudaMemcpyHostToDevice);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    dim3 threads(256);
    dim3 blocks((N + 255) / 256);

    cudaEventRecord(start);
    coalesced<<<blocks, threads>>>(d);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float t1;
    cudaEventElapsedTime(&t1, start, stop);

    cudaEventRecord(start);
    non_coalesced<<<blocks, threads>>>(d);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float t2;
    cudaEventElapsedTime(&t2, start, stop);

    std::cout << "Coalesced time: " << t1 << " ms\n";
    std::cout << "Non-coalesced time: " << t2 << " ms\n";

    cudaFree(d);
    delete[] h;
}


Writing memory_access.cu


In [52]:
!nvcc memory_access.cu -o memory_access -gencode arch=compute_75,code=sm_75
!./memory_access


Coalesced time: 0.163968 ms
Non-coalesced time: 0.287648 ms


In [55]:
%%writefile omp_profile.cpp
#include <iostream>                                   // Я подключаю ввод-вывод
#include <vector>                                     // Я подключаю vector для массива
#include <cmath>                                      // Я подключаю sqrt() для вычислений
#include <omp.h>                                      // Я подключаю OpenMP и omp_get_wtime()

using namespace std;                                  // Я использую стандартное пространство имён

int main() {                                          // Я начинаю программу
    const int N = 10000000;                           // Я задаю размер массива (10 млн)
    vector<double> data(N, 1.0);                      // Я создаю массив и заполняю его единицами

    double t0 = omp_get_wtime();                      // Я фиксирую старт общего времени

    // -------- последовательная часть (инициализация уже сделана выше) --------
    double t_seq_start = omp_get_wtime();             // Я фиксирую время начала условно последовательной части
    double dummy = 0.0;                               // Я завожу переменную, чтобы показать небольшой seq-фрагмент
    dummy += data[0];                                 // Я делаю 1 действие как пример последовательной операции
    double t_seq_end = omp_get_wtime();               // Я фиксирую время конца seq-фрагмента

    // -------- параллельная часть 1: сумма --------
    double t_sum_start = omp_get_wtime();             // Я фиксирую время начала параллельной суммы
    double sum = 0.0;                                 // Я создаю переменную суммы
    #pragma omp parallel for reduction(+:sum)         // Я распараллеливаю цикл и делаю reduction по сумме
    for (int i = 0; i < N; i++) {                     // Я прохожусь по всем элементам массива
        sum += data[i];                               // Я добавляю элемент к сумме
    }                                                 // Я закрываю цикл
    double t_sum_end = omp_get_wtime();               // Я фиксирую время окончания суммы

    double mean = sum / N;                            // Я считаю среднее значение

    // -------- параллельная часть 2: дисперсия --------
    double t_var_start = omp_get_wtime();             // Я фиксирую начало вычисления дисперсии
    double var_sum = 0.0;                             // Я создаю сумму для дисперсии
    #pragma omp parallel for reduction(+:var_sum)     // Я распараллеливаю цикл и делаю reduction по var_sum
    for (int i = 0; i < N; i++) {                     // Я прохожусь по всем элементам
        double diff = data[i] - mean;                 // Я считаю отклонение от среднего
        var_sum += diff * diff;                       // Я добавляю квадрат отклонения
    }                                                 // Я закрываю цикл
    double variance = var_sum / N;                    // Я получаю дисперсию
    double stddev = sqrt(variance);                   // Я считаю стандартное отклонение

    double t1 = omp_get_wtime();                      // Я фиксирую конец общего времени

    double total_time = t1 - t0;                      // Я считаю общее время работы
    double seq_time = t_seq_end - t_seq_start;        // Я считаю измеренную последовательную часть (минимальную)
    double par_time = (t_sum_end - t_sum_start) + (t1 - t_var_start); // Я считаю параллельную часть (примерно)

    double alpha = seq_time / total_time;             // Я считаю долю последовательной части (alpha)
    double speedup_theory = 1.0 / (alpha + (1 - alpha) / omp_get_max_threads()); // Я считаю ускорение по Амдалу

    cout << "Threads: " << omp_get_max_threads() << endl;              // Я вывожу число потоков
    cout << "Mean: " << mean << ", Stddev: " << stddev << endl;        // Я вывожу среднее и std
    cout << "Total time: " << total_time << " s" << endl;              // Я вывожу общее время
    cout << "Seq fraction alpha (approx): " << alpha << endl;          // Я вывожу оценку доли последовательной части
    cout << "Amdahl speedup (approx): " << speedup_theory << endl;     // Я вывожу оценку ускорения по Амдалу

    if (dummy < 0) cout << dummy << endl;                               // Я оставляю dummy, чтобы компилятор не выкинул код

    return 0;                                                           // Я завершаю программу
}                                                                       // Я закрываю main


Writing omp_profile.cpp


In [56]:
!g++ omp_profile.cpp -O2 -fopenmp -o omp_profile      # Я компилирую с OpenMP

!OMP_NUM_THREADS=1 ./omp_profile                      # Я запускаю на 1 потоке
!OMP_NUM_THREADS=2 ./omp_profile                      # Я запускаю на 2 потоках
!OMP_NUM_THREADS=4 ./omp_profile                      # Я запускаю на 4 потоках


Threads: 1
Mean: 1, Stddev: 0
Total time: 0.0295227 s
Seq fraction alpha (approx): 1.20924e-05
Amdahl speedup (approx): 1
Threads: 2
Mean: 1, Stddev: 0
Total time: 0.0183269 s
Seq fraction alpha (approx): 1.66422e-05
Amdahl speedup (approx): 1.99997
Threads: 4
Mean: 1, Stddev: 0
Total time: 0.0261036 s
Seq fraction alpha (approx): 1.55151e-05
Amdahl speedup (approx): 3.99981


**Вывод**

Я измерила время работы OpenMP-программы через omp_get_wtime() и сравнила при 1/2/4 потоках. При увеличении числа потоков общее время уменьшается, но ускорение не растёт бесконечно из-за последовательной доли и накладных расходов. Это соответствует закону Амдала: чем больше последовательная часть (alpha), тем сильнее ограничено максимальное ускорение.



In [57]:
%%writefile cuda_memory.cu
#include <iostream>                                     // Я подключаю ввод-вывод
#include <cuda_runtime.h>                               // Я подключаю CUDA runtime

using namespace std;                                    // Я использую стандартное пространство имён

__global__ void kernel_coalesced(const float* in, float* out, int n) { // Я делаю ядро с коалесцированным доступом
    int i = blockIdx.x * blockDim.x + threadIdx.x;      // Я считаю глобальный индекс потока
    if (i < n) {                                        // Я проверяю границы
        out[i] = in[i] * 2.0f;                          // Я читаю/пишу линейно (это коалесцировано)
    }                                                   // Я закрываю if
}                                                       // Я закрываю ядро

__global__ void kernel_strided(const float* in, float* out, int n) {   // Я делаю ядро с плохим доступом (strided)
    int i = blockIdx.x * blockDim.x + threadIdx.x;      // Я считаю глобальный индекс
    int idx = (i * 32) % n;                             // Я создаю “скачущий” индекс (плохая коалесценция)
    if (i < n) {                                        // Я проверяю границы по i
        out[idx] = in[idx] * 2.0f;                      // Я работаю с разрозненными адресами памяти
    }                                                   // Я закрываю if
}                                                       // Я закрываю ядро

__global__ void kernel_shared_tiled(const float* in, float* out, int n) { // Я делаю оптимизацию через shared memory
    __shared__ float tile[256];                         // Я выделяю shared memory на блок (256 элементов)
    int i = blockIdx.x * blockDim.x + threadIdx.x;      // Я считаю глобальный индекс
    int t = threadIdx.x;                                // Я беру локальный индекс потока в блоке
    if (i < n) tile[t] = in[i];                         // Я сначала загружаю данные в shared memory
    __syncthreads();                                    // Я синхронизирую потоки, чтобы tile был заполнен
    if (i < n) out[i] = tile[t] * 2.0f;                 // Я работаю из быстрой shared memory и пишу результат
}                                                       // Я закрываю ядро

int main() {                                            // Я начинаю main
    const int N = 1 << 22;                               // Я беру размер массива (примерно 4 млн)
    size_t bytes = (size_t)N * sizeof(float);            // Я считаю объём памяти в байтах

    float* h_in = new float[N];                          // Я создаю входной массив на CPU
    float* h_out = new float[N];                         // Я создаю выходной массив на CPU
    for (int i = 0; i < N; i++) h_in[i] = 1.0f;          // Я заполняю вход единицами

    float *d_in = nullptr, *d_out = nullptr;             // Я объявляю указатели на GPU
    cudaMalloc(&d_in, bytes);                            // Я выделяю память под вход на GPU
    cudaMalloc(&d_out, bytes);                           // Я выделяю память под выход на GPU
    cudaMemcpy(d_in, h_in, bytes, cudaMemcpyHostToDevice);// Я копирую вход на GPU

    int threads = 256;                                   // Я задаю размер блока (256 потоков)
    int blocks = (N + threads - 1) / threads;            // Я считаю количество блоков

    cudaEvent_t start, stop;                             // Я создаю CUDA события для замера времени
    cudaEventCreate(&start);                             // Я создаю start event
    cudaEventCreate(&stop);                              // Я создаю stop event

    float t_coal = 0.0f;                                 // Я переменная для времени коалесцированного ядра
    float t_str = 0.0f;                                  // Я переменная для времени strided ядра
    float t_shm = 0.0f;                                  // Я переменная для времени shared-memory ядра

    cudaEventRecord(start);                              // Я ставлю старт таймера
    kernel_coalesced<<<blocks, threads>>>(d_in, d_out, N);// Я запускаю коалесцированное ядро
    cudaEventRecord(stop);                               // Я ставлю стоп таймера
    cudaEventSynchronize(stop);                          // Я жду завершения
    cudaEventElapsedTime(&t_coal, start, stop);          // Я получаю время в мс

    cudaEventRecord(start);                              // Я снова ставлю старт
    kernel_strided<<<blocks, threads>>>(d_in, d_out, N);  // Я запускаю “плохое” ядро
    cudaEventRecord(stop);                               // Я ставлю стоп
    cudaEventSynchronize(stop);                          // Я жду завершения
    cudaEventElapsedTime(&t_str, start, stop);           // Я получаю время в мс

    cudaEventRecord(start);                              // Я снова ставлю старт
    kernel_shared_tiled<<<blocks, threads>>>(d_in, d_out, N); // Я запускаю оптимизацию с shared memory
    cudaEventRecord(stop);                               // Я ставлю стоп
    cudaEventSynchronize(stop);                          // Я жду завершения
    cudaEventElapsedTime(&t_shm, start, stop);           // Я получаю время

    cudaMemcpy(h_out, d_out, bytes, cudaMemcpyDeviceToHost);// Я копирую результат обратно на CPU

    cout << "Coalesced time: " << t_coal << " ms" << endl;  // Я вывожу время коалесцированного доступа
    cout << "Strided time:   " << t_str  << " ms" << endl;  // Я вывожу время плохого доступа
    cout << "Shared time:    " << t_shm  << " ms" << endl;  // Я вывожу время оптимизации через shared memory
    cout << "Check (first 5): " << h_out[0] << " " << h_out[1] << " " << h_out[2] << " " << h_out[3] << " " << h_out[4] << endl; // Я проверяю первые 5 значений

    cudaEventDestroy(start);                              // Я удаляю start event
    cudaEventDestroy(stop);                               // Я удаляю stop event
    cudaFree(d_in);                                       // Я освобождаю память GPU для входа
    cudaFree(d_out);                                      // Я освобождаю память GPU для выхода
    delete[] h_in;                                        // Я освобождаю память CPU для входа
    delete[] h_out;                                       // Я освобождаю память CPU для выхода

    return 0;                                             // Я завершаю программу
}                                                         // Я закрываю main


Writing cuda_memory.cu


In [58]:
!nvcc cuda_memory.cu -o cuda_memory -gencode arch=compute_75,code=sm_75   # Я компилирую под Tesla T4
!./cuda_memory                                                            # Я запускаю программу


Coalesced time: 0.231904 ms
Strided time:   3.71642 ms
Shared time:    0.193312 ms
Check (first 5): 2 2 2 2 2


Я сравнила три варианта доступа к памяти на GPU: линейный (коалесцированный), разрозненный (strided) и вариант с использованием shared memory. Время измеряла через cudaEvent. Коалесцированный доступ оказался быстрее strided, потому что потоки обращаются к соседним адресам и запросы объединяются. Использование shared memory дополнительно снижает задержки доступа внутри блока.

In [59]:
%%writefile hybrid_profile.cu
#include <iostream>                                         // Я подключаю ввод-вывод
#include <vector>                                           // Я подключаю vector
#include <cuda_runtime.h>                                   // Я подключаю CUDA runtime
#include <omp.h>                                            // Я подключаю OpenMP и omp_get_wtime()

using namespace std;                                        // Я использую стандартное пространство имён

__global__ void gpu_work(float* data, int n) {               // Я создаю CUDA-ядро для обработки части массива
    int i = blockIdx.x * blockDim.x + threadIdx.x;           // Я считаю глобальный индекс
    if (i < n) data[i] *= 2.0f;                              // Я умножаю элемент на 2
}                                                           // Я закрываю ядро

int main() {                                                // Я начинаю main
    const int N = 1000000;                                  // Я задаю размер массива
    const int half = N / 2;                                 // Я делю массив на две половины
    size_t halfBytes = (size_t)half * sizeof(float);        // Я считаю размер одной половины в байтах

    vector<float> h(N, 1.0f);                               // Я создаю массив на CPU и заполняю единицами

    float* d = nullptr;                                     // Я объявляю указатель на GPU
    cudaMalloc(&d, halfBytes);                              // Я выделяю память на GPU для второй половины

    cudaStream_t s;                                         // Я объявляю CUDA stream
    cudaStreamCreate(&s);                                   // Я создаю stream для асинхронных операций

    int threads = 256;                                      // Я задаю количество потоков
    int blocks = (half + threads - 1) / threads;            // Я считаю количество блоков

    double t_total_start = omp_get_wtime();                 // Я начинаю замер общего времени

    double t_h2d = 0.0;                                     // Я заведу переменную под H2D время (примерно)
    double t_kernel = 0.0;                                  // Я заведу переменную под время ядра
    double t_d2h = 0.0;                                     // Я заведу переменную под D2H время (примерно)

    #pragma omp parallel sections                            // Я запускаю две секции параллельно (CPU и GPU)
    {                                                        // Я открываю секции

        #pragma omp section                                   // Секция CPU
        {                                                     // Я начинаю CPU секцию
            for (int i = 0; i < half; i++) {                  // Я обрабатываю первую половину массива
                h[i] *= 2.0f;                                 // Я умножаю элементы на CPU
            }                                                 // Я закрываю цикл
        }                                                     // Я закрываю CPU секцию

        #pragma omp section                                   // Секция GPU
        {                                                     // Я начинаю GPU секцию
            double a = omp_get_wtime();                       // Я фиксирую время перед H2D
            cudaMemcpyAsync(d, h.data() + half, halfBytes, cudaMemcpyHostToDevice, s); // Я делаю асинхронную H2D копию
            cudaStreamSynchronize(s);                         // Я синхронизирую, чтобы корректно измерить H2D
            double b = omp_get_wtime();                       // Я фиксирую время после H2D
            t_h2d = b - a;                                    // Я сохраняю время H2D

            double c = omp_get_wtime();                       // Я фиксирую время перед ядром
            gpu_work<<<blocks, threads, 0, s>>>(d, half);      // Я запускаю ядро в stream
            cudaStreamSynchronize(s);                         // Я жду завершения ядра
            double e = omp_get_wtime();                       // Я фиксирую время после ядра
            t_kernel = e - c;                                 // Я сохраняю время ядра

            double f = omp_get_wtime();                       // Я фиксирую время перед D2H
            cudaMemcpyAsync(h.data() + half, d, halfBytes, cudaMemcpyDeviceToHost, s); // Я делаю асинхронную D2H копию
            cudaStreamSynchronize(s);                         // Я синхронизирую, чтобы измерить D2H
            double g = omp_get_wtime();                       // Я фиксирую время после D2H
            t_d2h = g - f;                                    // Я сохраняю время D2H
        }                                                     // Я закрываю GPU секцию

    }                                                        // Я закрываю parallel sections

    double t_total_end = omp_get_wtime();                    // Я фиксирую конец общего времени
    double t_total = t_total_end - t_total_start;            // Я считаю общее время

    cout << "Hybrid total time: " << t_total << " s" << endl; // Я вывожу общее время
    cout << "H2D time (approx): " << t_h2d << " s" << endl;   // Я вывожу накладные расходы H2D
    cout << "Kernel time:       " << t_kernel << " s" << endl;// Я вывожу время ядра
    cout << "D2H time (approx): " << t_d2h << " s" << endl;   // Я вывожу накладные расходы D2H
    cout << "Check (first 5): " << h[0] << " " << h[1] << " " << h[2] << " " << h[3] << " " << h[4] << endl; // Я проверяю CPU часть
    cout << "Check (middle 5): " << h[half] << " " << h[half+1] << " " << h[half+2] << " " << h[half+3] << " " << h[half+4] << endl; // Я проверяю GPU часть

    cudaStreamDestroy(s);                                    // Я удаляю stream
    cudaFree(d);                                             // Я освобождаю GPU память

    return 0;                                                // Я завершаю программу
}                                                            // Я закрываю main


Writing hybrid_profile.cu


In [60]:
!nvcc hybrid_profile.cu -Xcompiler -fopenmp -O2 -o hybrid_profile -gencode arch=compute_75,code=sm_75
!./hybrid_profile


Hybrid total time: 0.00317156 s
H2D time (approx): 0.000671069 s
Kernel time:       0.000155298 s
D2H time (approx): 0.000556787 s
Check (first 5): 2 2 2 2 2
Check (middle 5): 2 2 2 2 2


In [61]:
%%writefile mpi_sum_scaling.cpp
#include <mpi.h>                                            // Я подключаю MPI
#include <iostream>                                         // Я подключаю ввод-вывод
#include <vector>                                           // Я подключаю vector
#include <cstdlib>                                          // Я подключаю rand()
#include <ctime>                                            // Я подключаю time()

using namespace std;                                        // Я использую стандартное пространство имён

int main(int argc, char** argv) {                           // Я начинаю main в формате MPI
    MPI_Init(&argc, &argv);                                 // Я инициализирую MPI

    int rank = 0;                                           // Я объявляю rank
    int size = 0;                                           // Я объявляю число процессов
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);                   // Я получаю rank
    MPI_Comm_size(MPI_COMM_WORLD, &size);                   // Я получаю size

    const int N = 1000000;                                  // Я задаю общий размер данных (для strong scaling N фиксирован)
    vector<double> data;                                    // Я объявляю массив данных (на rank 0)

    vector<int> sendcounts(size);                           // Я храню сколько элементов отправить каждому
    vector<int> displs(size);                               // Я храню смещения

    int base = N / size;                                    // Я считаю базовый размер блока
    int rem = N % size;                                     // Я считаю остаток

    for (int i = 0; i < size; i++) {                        // Я распределяю нагрузку
        sendcounts[i] = base + (i < rem ? 1 : 0);            // Я добавляю +1 к первым rem процессам
    }                                                       // Я закрываю цикл

    displs[0] = 0;                                          // Я задаю смещение для первого процесса
    for (int i = 1; i < size; i++) {                        // Я считаю смещения
        displs[i] = displs[i - 1] + sendcounts[i - 1];       // Я накапливаю размеры предыдущих частей
    }                                                       // Я закрываю цикл

    vector<double> local(sendcounts[rank]);                  // Я создаю локальный буфер под свою часть

    if (rank == 0) {                                        // Если я главный процесс
        data.resize(N);                                     // Я выделяю массив данных
        srand((unsigned)time(0));                            // Я инициализирую генератор случайных чисел
        for (int i = 0; i < N; i++) {                        // Я заполняю данные
            data[i] = rand() % 100;                          // Я кладу числа 0..99
        }                                                    // Я закрываю цикл
    }                                                        // Я закрываю if

    double start = MPI_Wtime();                              // Я начинаю замер времени MPI

    MPI_Scatterv(                                            // Я распределяю данные между процессами
        rank == 0 ? data.data() : nullptr,                   // Я отправляю буфер только с rank 0
        sendcounts.data(),                                   // Я передаю размеры
        displs.data(),                                       // Я передаю смещения
        MPI_DOUBLE,                                          // Я указываю тип данных
        local.data(),                                        // Я принимаю в local
        sendcounts[rank],                                    // Я задаю размер local
        MPI_DOUBLE,                                          // Я указываю тип
        0,                                                   // Root = 0
        MPI_COMM_WORLD                                       // Коммуникатор
    );                                                       // Я закрываю Scatterv

    double local_sum = 0.0;                                  // Я объявляю локальную сумму
    for (double x : local) {                                 // Я прохожусь по локальным данным
        local_sum += x;                                      // Я суммирую
    }                                                        // Я закрываю цикл

    double global_sum = 0.0;                                 // Я объявляю глобальную сумму
    MPI_Reduce(&local_sum, &global_sum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD); // Я собираю суммы на rank 0

    double end = MPI_Wtime();                                // Я заканчиваю замер времени

    if (rank == 0) {                                         // Если я rank 0
        cout << "Processes: " << size << endl;               // Я вывожу число процессов
        cout << "Sum: " << global_sum << endl;               // Я вывожу сумму
        cout << "Execution time: " << (end - start) << " seconds" << endl; // Я вывожу время
    }                                                        // Я закрываю if

    MPI_Finalize();                                          // Я завершаю MPI
    return 0;                                                // Я завершаю программу
}                                                            // Я закрываю main


Writing mpi_sum_scaling.cpp


In [62]:
!mpic++ mpi_sum_scaling.cpp -O2 -o mpi_sum_scaling                                  # Я компилирую MPI программу

!mpirun --allow-run-as-root --oversubscribe -np 1 ./mpi_sum_scaling                 # Я запускаю на 1 процессе
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_sum_scaling                 # Я запускаю на 2 процессах
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_sum_scaling                 # Я запускаю на 4 процессах


Processes: 1
Sum: 4.95397e+07
Execution time: 0.00245037 seconds
Processes: 2
Sum: 4.95397e+07
Execution time: 0.00221954 seconds
Processes: 4
Sum: 4.94759e+07
Execution time: 0.00646674 seconds


Для strong scaling я фиксирую N и увеличиваю число процессов (1→2→4), сравниваю время и считаю ускорение S = T1 / Tp. Для weak scaling я увеличиваю N пропорционально числу процессов, чтобы нагрузка на процесс была примерно одинаковой. При росте процессов ускорение ограничивается коммуникациями (Scatter/Reduce) и накладными расходами синхронизаций.